In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
import itertools
import random

from utils import *

In [4]:
df = load_process_data('features.csv')
df.head()

,song,bpm,energy,key,chroma_0,chroma_1,chroma_2,chroma_3,chroma_4,chroma_5,...,mfcc_11,mfcc_12,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,spectral_centroids
0,Freeze Corleone - Freeze Raël (ØKKY Techno edi...,160,1.865880,Bm,-0.477682,1.012120,2.647437,1.325615,0.640637,0.973365,...,-0.937401,1.563838,0.129216,1.034066,-0.103144,-0.827386,-0.851569,-0.288211,-0.203327,0.021286
1,Matrakk - 69 Carats [UFR001].mp3,155,-0.289873,F#m,-0.731253,-0.801464,-0.011158,-0.945499,-1.100119,-0.152173,...,1.572870,-0.554078,-0.041583,-0.247208,0.839230,2.461730,1.959794,0.790399,-1.096800,-0.675643
2,Ascendant Vierge - Influenceur (Nyctonian Indu...,160,-1.124974,Bm,-0.194831,-0.734684,1.682649,0.368762,-0.451555,-0.840985,...,1.039487,0.346685,0.029431,0.180843,-0.170574,0.363196,-0.480722,-0.209095,-1.125778,-1.048123
3,Oliver Tree - Miss You (Nyctonian Tech Pop Te...,155,-1.578871,F#m,-0.028171,-0.618037,1.767263,-0.031793,0.927185,-0.157011,...,-1.312218,-0.155950,-0.611812,-1.018009,0.399838,-0.457980,-1.062526,-0.330170,-0.464136,-1.106900
4,Jaëss - Kick Dans Ton Cor.mp3,155,0.744366,C#m,-1.872866,-1.324882,-0.865519,-0.215081,2.823866,-0.163598,...,2.255093,1.250315,1.717135,-0.247022,1.017935,0.029487,1.380585,0.856681,1.449897,1.124124


In [6]:
X = df.drop(['song', 'bpm', 'key'], axis=1).values

In [7]:
# PCA to plot the data in 2D (using plotly)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Keep a column with color that change with bpm increasing (bpm being the first column of X)
X_pca = np.c_[X_pca, df['bpm'].values]

# Plot the data in 2D
fig = px.scatter(x=X_pca[:, 0], y=X_pca[:, 1], color=X_pca[:, 2])
fig.show()


In [9]:
# 'Pier - Angèle Saiyan (Techno Edit).mp3" bpm is too low, we remove it
df = df[df.song != 'Pier - Angèle Saiyan (Techno Edit).mp3']

# Assuming df is your DataFrame with songs and their BPMs
G = create_graph(df)
G = create_graph_key_constraint(df)

# Normalize BPM for color mapping
bpm_normalized = (df.set_index('song')['bpm'] - df['bpm'].min()) / (df['bpm'].max() - df['bpm'].min())

# Calculate node positions
pos = nx.spring_layout(G)

# Node trace
node_trace = go.Scatter(
    x=[pos[node][0] for node in G.nodes()],
    y=[pos[node][1] for node in G.nodes()],
    mode='markers',
    marker=dict(
        size=10,
        color=[bpm_normalized[node] for node in G.nodes()],
        colorscale='Viridis',
        colorbar=dict(title='BPM'),
        line_width=2,
        opacity=0.8  # Reduced opacity
    ),
    text=[node for node in G.nodes()],  # Hover text
    hoverinfo='text'
)

# Edge trace
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)

# Create the figure
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title='Song Graph',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

fig.show()

In [ ]:
def find_shortest_hamiltonian_path(G):
    # Store the shortest path and its length
    shortest_path = None
    shortest_path_length = float('inf')

    # Iterate over all permutations of nodes to check each possible path
    for path in itertools.permutations(G.nodes()):
        current_length = 0
        valid_path = True

        # Calculate the length of the current path
        for i in range(len(path) - 1):
            if G.has_edge(path[i], path[i + 1]):
                current_length += G[path[i]][path[i + 1]]['weight']
            else:
                valid_path = False
                break

        # Update the shortest path if a shorter one is found
        if valid_path and current_length < shortest_path_length:
            shortest_path_length = current_length
            shortest_path = path

        print(f"Checked path: {path}, Length: {current_length}")

    return shortest_path, shortest_path_length


path, length = find_shortest_hamiltonian_path(G)
print(f"Shortest Hamiltonian path: {path}, Length: {length}")